Math shared by all implementations

All implementations have the fundamental bits:

input matrix X of size [num_samples, num_features]

output matrix y of size [num_samples, num_outputs]

projection matrix W of size [num_features, num_neurons]

non-linear tranformation function ufunc like hyperbolic tangent (np.tanh), sigmoid, or any other

output weights B of size [num_neurons, num_outputs]

optional output bias/intercept of size [num_outputs, ]


In [381]:
import numpy as np
import pandas  as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from scipy import linalg
from math import sqrt

In [382]:
class clientClass:
   
    def __init__(self,client_id,batch_size,h_neurons=20, alpha=0.001):
        
        self.i=client_id
        cl= np.load(f"client{self.i}.npz")

        if client_id == 1:
            n = 100
        elif client_id == 2:
            n = 1000
        else:
            n = 10000
            
        
        X_t = cl[f"X_a{self.i}_sc"][:n]
        X_s = cl[f"X_a{self.i}_sc"][n:]
        y_t = cl[f"y_a{self.i}"][:n]
        y_s = cl[f"y_a{self.i}"][n:]

           
        self.input_size = len(X_t)
        self.n_features = len(X_t[1])
        self.h_neurons=h_neurons
        self.w = None
        self.b = None
        self.batch_size=batch_size
        self.reg_term = np.eye(self.h_neurons) * alpha
        self.X_t = X_t
        self.X_s = X_s
        self.y_t = y_t
        self.y_s = y_s
          
            
    def get_Nfs(self):
        return self.input_size, self.n_features
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b  
    
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        if self.batch_size >= self.input_size:
            batch_X.append(self.X_t)
            batch_y.append(self.y_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(self.X_t[i:i + self.batch_size-1])
                batch_y.append(self.y_t[i:i + self.batch_size-1])
            
        for X,y in zip(batch_X,batch_y): 
            
             # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)

            HTH = H.T @ H + self.reg_term
            HTy = H.T @ y
              
            yield HTH, HTy               
   
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(self.X_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(self.y_s, pred_test)
        return r2


In [383]:


def par_B(cl1,cl2,cl3):
    # Perform some operation using a for loop
    # combine some client data
    clients = [cl1, cl2, cl3]
    HtH1 = None
    HtY1 = None
    Bd1={}
    B1=[]
    Bd=[]

    for client in clients:
        sample, c_nf= client.get_Nfs()
        w_inital = np.random.rand(c_nf, hidden_size)
        bias_inital = np.random.rand(hidden_size)
        #print(w_inital)

        client.set_W_b(w_inital, bias_inital)
        data = client.batches()

        for hth, hty in data:

            HtH1 = hth
            HtY1 = hty

            #print(f"Client   : {client.get_Nfs()}")
            Bd.append(np.linalg.solve(HtH1, HtY1))
            #print(f"Bd: {Bd}")
        Bd1[client.i] = Bd 
        Bd=[]

    return Bd1

In [384]:
def average_matrices_Sl(matrix1):
    # Check if the matrices have the same shape
    if len(matrix1) == 0:
        return None  # Handle the case of an empty list
    average_matrix = sum(matrix1 ) / len(matrix1)
    return average_matrix
    

In [385]:
# I am a server now

# initialize ELM parameters shared among clients

hidden_size=20


# initialize clients, server sends ELM parameters to clients
cl1 = clientClass(1, batch_size=100, h_neurons=hidden_size, alpha=0.001)
cl2 = clientClass(2, batch_size=300, h_neurons=hidden_size, alpha=0.001)
cl3 = clientClass(3, batch_size=1000, h_neurons=hidden_size, alpha=0.001)

#cl1_input_size,cl1_n_features =cl1.get_Nfs()
#cl2_input_size,cl2_n_features =cl2.get_Nfs()
#cl3_input_size,cl3_n_features =cl3.get_Nfs()

input_size=8
w_inital = np.random.rand(input_size, hidden_size)
bias_inital = np.random.rand(hidden_size)

clients = [cl1, cl2, cl3]
cl1.set_W_b(w_inital, bias_inital)
cl2.set_W_b(w_inital, bias_inital)
cl3.set_W_b(w_inital, bias_inital)


In [386]:
B_1i=par_B(cl1,cl2,cl3)

In [387]:
for i in range(1,len(B_1i)+1):
    print(f"Client_{i}")
    print(f"{average_matrices_Sl(B_1i[i])}"  )

Client_1
[ 0.0018309   0.27366645 -0.18269487  1.41177489  0.88816717  0.6132533
 -0.37452951  0.368464   -1.15809518 -0.35156554 -0.72861862 -0.3855515
 -0.31425373 -0.39655482  0.43294479  0.16527108  0.18453703 -0.11564701
  0.31746233 -0.02950896]
Client_2
[ 0.78123382 -0.15824864 -0.28775314 -0.18833118  1.2281139  -0.18161286
  0.95770778 -0.50357389 -1.27350353 -2.36275568 -1.54712132  0.11531156
  0.38576782  0.26970708 -0.17034188 -0.61024616 -0.32149112  0.31275882
  1.51808734  0.32527513]
Client_3
[ 1.8340578  -0.36727843 -0.13997193 -0.38703397  0.01855467 -0.1704152
 -0.47306788  1.37956307  0.03873956  1.06054508 -0.29100606 -0.15156306
 -0.05915995 -0.3641569  -2.16116788  0.70998747 -0.70654386 -0.79166136
 -0.29166051 -0.04094877]


In [388]:
# combine some client data

B11=[]
for hth, hty in cl1.batches():
    print("next batch")
    
    HtH11 = hth
    HtY11 = hty
    B11.append(np.linalg.solve(HtH11, HtY11)) 
    # ... ask client to compute their performance ....
    #r2_cl1 = cl1.predict_test(B1)

next batch


In [389]:
B22=[]

for hth, hty in cl2.batches():
    print("next batch")
    HtH22 = hth
    HtY22 = hty
    B22.append(np.linalg.solve(HtH22, HtY22))
    # ... ask client to compute their performance ....

     #r2_cl2 = cl1.predict_test(B2)

next batch
next batch
next batch
next batch


In [390]:
B33 = []
for hth, hty in cl3.batches():
    print("next batch")
    HtH33 = hth
    HtY33 = hty
  
    B33.append(np.linalg.solve(HtH33, HtY33))
   
    # ... ask client to compute their performance ....
     #r2_cl3 = cl1.predict_test(B3)




next batch
next batch
next batch
next batch
next batch
next batch
next batch
next batch
next batch
next batch


In [391]:
average_matrices_Sl(B_1i[1])

array([ 0.0018309 ,  0.27366645, -0.18269487,  1.41177489,  0.88816717,
        0.6132533 , -0.37452951,  0.368464  , -1.15809518, -0.35156554,
       -0.72861862, -0.3855515 , -0.31425373, -0.39655482,  0.43294479,
        0.16527108,  0.18453703, -0.11564701,  0.31746233, -0.02950896])